1. Crie uma MLP genérica para modelo de classificação binária (ou não) implementando forward, backward e loss.
    
    a. É necessário que o usuário possa definir quantas layers e neurônios desejar
    
    b. É necessário que a loss utilizada seja binary cross-entropy (estude um pouco mais sobre esse tópico)
    
    c. É necessário que a rede funcione para diferentes datasets de classificação, considerando que o dado de entrada para o modelo já esteja no formato desejado, como por exemplo número de features, target binário.
    
    d. É necessário que a  função de ativação da layer de output retorne uma probabilidade (entre 0 e 1), então use algo como sigmoid por exemplo.
    
    e. A regra de update dos pesos pode ser feita utilizando a regra básica do gradient descent (weight = weight - learning_rate * gradient)
2. (Opcional - Difícil 1) Crie uma MLP para classificar o dataset MNIST e faça as adaptações necessárias
3. (Opcional - Difícil 2) Crie um otimizador para update dos pesos utilizando o gradient e learning_rate. Como por exemplo, o otimizador Adam.
As informações da aula de hoje estão dentro do repositório de lectures/users/luisa/NN/XOR-zero.ipynb. Se precisarem das informações escritas posso emprestar meus rascunhos.


Defining generic Layer class

In [1]:
import numpy as np
import math

class NetOperation():

  def forward():
    pass

  def backward():
    pass

  def optmize():
    pass

class Layer(NetOperation):
  def __init__(self, n_in: int, n_out: int):
    self.weights = np.random.randn(n_in, n_out) * 0.01
    self.bias = np.zeros(shape=(1, n_out))

  # z = a1w1 + a2w2 + a3w3 + ... + b
  def forward(self, a):
    self.a = a
    return a.dot(self.weights) + self.bias

  # dgrad = dL / z
  # dL / dwi = dL / dz * dz / dwi
  def backward(self, grad):
    self.grad_weights = self.a.T.dot(grad)
    self.grad_bias = np.sum(grad, axis= 0)
    # DUVIDA PRIMORDIAL (PRECISO VER ISSO, NÃO ENTENDI MUITO BEM)
    # peguei do código da luiza a ideia
    return grad.dot(self.weights.T)

  def optimize(self, learning_rate):
    self.weights -= self.grad_weights * learning_rate
    self.grad_weights = None

    self.bias -= self.grad_bias * learning_rate
    self.grad_bias = None

class ReLU(NetOperation):
  # a: activation
  # z: weighted sum

  # a = f(z)
  def f(z):
    return np.maximum(0, z)
  
  # da / dz
  # a = df(z)
  def df(z):
    return np.greater(0, z).astype(int)

  def forward(self, z):
    self.z = z
    return self.f(z)
  
  # dgrad = dL / da
  # dL / dz = da / dz * dL / da
  def backward(self, grad):
    return self.df(self.z) * grad
  
class Sigmoid(NetOperation):
  # a: activation
  # z: weighted sum

  def f(z):
    return 1 / (1 + math.exp(-z))
  
  def df(self, z):
    s = self.f(z)
    return s * (1 - s)

  def forward(self, z):
    self.z = z
    return self.f(z)
  
  def backward(self, grad):
    return self.df(self.z) * grad
    

In [2]:
class MLP(NetOperation):
  layers: list[NetOperation]

  def __init__(self, n_features: int, neurons_per_layer: list[int]):
    input_layer = [Layer(n_features, neurons_per_layer[0])]
    hidden_layers = []

    for i in range(len(neurons_per_layer)):
    
      hidden_layers.extend([ReLU(), Layer(neurons_per_layer[i], neurons_per_layer[i + 1])])


    self.layers = input_layer + hidden_layers + [Sigmoid()]


  def forward(self, x):
    activation = x

    for layer in self.layers:
      activation = layer.forward(activation)

    return activation
  
  # [Sigmoid(), Layer, ReLU, Layer, ReLU, Layer]
  # propagate error
  def backward(self, grad):
    self.layers.reverse()
    
    for layer in self.layers:
      grad = layer.backward(grad)

    self.layers.reverse()


Train

In [3]:
from matplotlib import pyplot as plt

def BCELoss(prediction, target):
  n = len(target)
  return - 1 / n * (target * np.log(prediction) + (1 - target) * np.log(1 - prediction))


X = np.array([
  [0, 0],
  [0, 1],
  [1, 0],
  [1, 1],
])
Y = np.array([
  [0],
  [1],
  [1],
  [0]
])

net = MLP(2, [2, 2, 1])

learning_rate = 0.1
batch_size = 2
xs = range(30)
accuracies = []
for epoch in xs:
  print(f'Epoch {epoch}')
  hits = 0
  for i in range(0, len(X), batch_size):
    train_sample = X[i]
    target_sample = Y[i]


    prediction = net.forward(train_sample)
    error = (prediction - target_sample)**2

    net.backward(2 * (prediction - target_sample))
    net.optimize(learning_rate)

    hits += prediction == target_sample

  accuracies.append((hits / len(X)).reshape(-1)[0])

  print(f'Accuracy: {hits / len(X)}')
plt.plot(xs, accuracies)


Epoch 0


NameError: name 'train' is not defined

Test

In [ ]:
hits = 0
for i in range(len(test['data'])):
  test_sample = test['data'][i].reshape(1, 4)
  target_sample = test['target'][i]

  prediction = layer.forward(test_sample)
  prediction = prediction >= 0.5

  hits += prediction == target_sample
print('Accuracy: ', hits/len(test['data']))

Accuracy:  [[1.]]
